In [76]:
import os

data_dir = "../interim/interim_1"
datasets = os.listdir(data_dir)

In [77]:
import re
def preprocess_text(text):
    # text = text.replace('-'," - ")
    # text = text.replace('\''," ' ")
    for symbol in ['-','\'','(',')','/',',','+','"','.']:
        text = text.replace(symbol,f" {symbol} ")
    text = re.sub(r"\s+",' ',text).strip()
    return text

In [78]:
polarity_dict = {
    "positive" : "POS",
    "negative" : "NEG",
    "neutral" : "NEU"
}
def transform_to_num_target(text,targets):
    result_num_targets = []
    splitted_text = text.split()
    for target in targets:
        aspect_term, aspect_category, polarity, opinion_term = target
        aspect_term = preprocess_text(aspect_term)
        opinion_term = preprocess_text(opinion_term)
        splitted_aspect_term = aspect_term.split()
        splitted_opinion_term = opinion_term.split()
        aspect_resulting_index = []
        opinion_resulting_index = []
        if aspect_term != "NULL":
            for i in range(0,len(splitted_text)-len(splitted_aspect_term)+1):
                window = splitted_text[i:i+len(splitted_aspect_term)]
                if ' '.join(window) == aspect_term:
                    aspect_resulting_index = [i+j for j in range(len(splitted_aspect_term))]
                    break
        else:
            aspect_resulting_index = [-1]
        for i in range(0,len(splitted_text)-len(splitted_opinion_term)+1):
            window = splitted_text[i:i+len(splitted_opinion_term)]
            if ' '.join(window) == opinion_term:
                opinion_resulting_index = [i+j for j in range(len(splitted_opinion_term))]
                break
        try:
            polarity = polarity_dict[polarity]
        except Exception as e:
            print(polarity)
            raise e
        if len(aspect_resulting_index) == 0 or len(opinion_resulting_index) == 0:
            raise Exception(f"Resulting index cannot be blank, text : {text} | aspect : {aspect_term} | opinion : {opinion_term}")
        result_num_targets.append((aspect_resulting_index,aspect_category,polarity,opinion_resulting_index))
    return result_num_targets

In [79]:
sample = "The food however , is what one might expect .####[['food', 'food quality', 'negative', 'expect']]"
text, targets = sample.split("####")
targets = eval(targets)

In [80]:
transform_to_num_target(text,targets)

[([1], 'food quality', 'NEG', [8])]

In [81]:
output_dir = "../interim"
data_split = ["train.txt","dev.txt","test.txt"]
for ds in datasets:
    for split in data_split:
        result_data = []
        path = os.path.join(data_dir,ds,split)
        print(path)
        output_path = os.path.join(output_dir,ds)
        if not os.path.exists(output_path):
            os.makedirs(output_path)
        with open(path,'r') as reader:
            data = reader.read().strip().split('\n')
        for line in data:
            try:
                text, targets = line.split("####")
            except Exception as e:
                print(line)
                raise e
            text = preprocess_text(text)
            targets = eval(targets)
            targets = transform_to_num_target(text,targets)
            result_data.append(f"{text}####{targets}")
        with open(os.path.join(output_path,split),'w') as writer:
            for line in result_data:
                writer.write(line + '\n')

../interim/interim_1\rest15\train.txt
../interim/interim_1\rest15\dev.txt
../interim/interim_1\rest15\test.txt
../interim/interim_1\rest16\train.txt
../interim/interim_1\rest16\dev.txt
../interim/interim_1\rest16\test.txt
